In [1]:
# Import the libraries

import pandas as pd
import numpy as np

# Decision Trees

`Decision Tree Algorithm` is a non-parametric supervised learning method used for classification and regression tasks. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.  
  
There are many different algorithms used to implement decision trees. In this notebook, we will implement the most basic one, which is called [ID3](https://en.wikipedia.org/wiki/ID3_algorithm). It is a greedy algorithm that builds a decision tree by selecting the best attribute that yields the highest information gain for each node.

## Data Preprocessing

Let's start with loading the training datasets into a pandas dataframe

In [2]:
df = pd.read_csv('car_train.csv')
df.head()

,Buying Price,Maintenance Price,Number of Doors,Capacity,Size of Luggage Boot,Estimated Safety,Decision
0,low,low,3,2,small,high,unacc
1,low,high,4,2,big,high,unacc
2,vhigh,low,3,4,small,high,acc
3,vhigh,med,5more,more,small,med,unacc
4,low,vhigh,4,more,med,high,acc


In [3]:
class Node:
    def __init__(self, dataset: pd.DataFrame, features: list, target: str):
        """Initializes the Node class. This class represents the current state of the dataset after a split has been made.
        """
        self.dataset = dataset
        self.features = features
        self.target = target

        self.best_feature = None
        self.best_feature_values = {}

        self.class_names = dataset[target].unique().tolist()
        self.target_probabilities = self.calculate_target_probabilities(self.dataset)
        self.entropy = self.calculate_entropy(self.dataset)
    
    def calculate_target_probabilities(self, dataset: pd.DataFrame) -> np.ndarray:
        """This function calculates the probabilities of each unique value in the target variable.
        """
        target_probabilties = dataset[self.target].value_counts(normalize=True).values
        return target_probabilties
    
    def calculate_entropy(self, dataset: pd.DataFrame) -> float:
        """Calculates the entropy of the target variable in the given dataset
        """
        target_probabilities = dataset[self.target].value_counts(normalize=True).values
        target_log_probabilities = np.log2(target_probabilities)

        entropy = -np.sum(target_probabilities * target_log_probabilities)
        return entropy

    def calculate_information_gain(self, feature: str) -> float:
        """Calculates the information gain for the gievn feature
        """

        # Find the unique values of the feature
        feature_values = self.dataset[feature].unique()

        # Find the probabilities of each unique value
        feature_probabilities = self.dataset[feature].value_counts(normalize=True).values

        # Find the entropy of each unique value
        feature_entropies = []
        for feature_value in feature_values:
            feature_subset = self.dataset[self.dataset[feature] == feature_value]
            feature_subset_entropy = self.calculate_entropy(feature_subset)
            feature_entropies.append(feature_subset_entropy)

        # Find the weighted average of the entropies
        feature_entropies = np.array(feature_entropies)
        weighted_average = np.sum(feature_probabilities * feature_entropies)

        # Find the information gain
        information_gain = self.entropy - weighted_average
        return information_gain

    def find_best_feature(self) -> str:
        """Finds the best feature which gives the highest information gain. This is the feature which we will use to split the dataset
        """
        # Get information gain for all features and select the one with the highest information gain
        information_gains = []
        for feature in self.features:
            information_gain = self.calculate_information_gain(feature)
            information_gains.append(information_gain)

        information_gains = np.array(information_gains)
        best_feature_index = np.argmax(information_gains)
        best_feature = self.features[best_feature_index]
        return best_feature
    
    def is_pure(self) -> bool:
        """Checks if the node is pure. A node is pure if all the target values are the same
        """
        return self.entropy == 0

    def predict(self) -> str:
        """Predicts the class of the current node. This is the class which has the highest probability
        """
        idx = np.argmax(self.target_probabilities)
        prediction = self.class_names[idx]
        return prediction

In [4]:
class Tree:
    def __init__(self, max_depth=None):
        self.root = None
        self.max_depth = max_depth
    
    def fit(self, dataset: pd.DataFrame, features: list, target: str, node: Node=None, depth: int=1):
        """Trains the decision tree model on the given dataset
        """

        if node is not None and node.is_pure(): # Do not split if the node is pure
            return
        
        if self.max_depth is not None and depth > self.max_depth: # Do not split if the max depth is reached
            return
        
        if not features: # Do not split if there are no features left
            return
        
        if node is None:
            node = Node(dataset, features, target)
            self.root = node
        
        # Find the best feature to split on
        best_feature = node.find_best_feature()
        node.best_feature = best_feature # Set the best feature of the node
        best_feature_values = dataset[best_feature].unique() # Find the unique values of the best feature

        for best_feature_value in best_feature_values:

            # Create a subset of the dataset which contains only the current best feature value and remove the best feature from the dataset
            #! NOTE: Remember to create a copy of the dataset while removing the feature. Otherwise, the original dataset will be modified
            best_feature_subset = dataset[dataset[best_feature] == best_feature_value]
            best_feature_subset = best_feature_subset.drop(columns=[best_feature])

            # Create a subset of the features which does not contain the best feature
            best_feature_subset_features = list(best_feature_subset.columns)
            best_feature_subset_features.remove(target)

            # Create a new node for the best split
            best_feature_subset_root = Node(best_feature_subset, best_feature_subset_features, target)
            node.best_feature_values[best_feature_value] = best_feature_subset_root


            # Recursively fit the model on the best feature subset
            self.fit(best_feature_subset, best_feature_subset_features, target, node=best_feature_subset_root, depth=depth+1)
        
    def predict(self, features: pd.Series) -> tuple:
        """Predict the class for the given features using the trained model
        """
        node = self.root
        decisions = []

        while node.best_feature is not None:
            decisions.append(node.best_feature)
            feature_value = features[node.best_feature]
            if feature_value not in node.best_feature_values:
                break
            node = node.best_feature_values[feature_value]
        
        prediction = node.predict()
        return prediction, decisions

In [5]:
features = df.columns.tolist()
features.remove('Decision')
target = 'Decision'

Create an instance of the tree

In [6]:
clf = Tree()

Fit the tree to the training data

In [7]:
clf.fit(df, features, target)

## Predictions on the Test Set

In [8]:
df_test = pd.read_csv('car_test.csv')
df_test.head()

,Buying Price,Maintenance Price,Number of Doors,Capacity,Size of Luggage Boot,Estimated Safety,Decision
0,low,vhigh,2,more,med,high,acc
1,vhigh,high,2,4,big,high,unacc
2,high,med,2,2,small,med,unacc
3,vhigh,med,3,2,big,med,unacc
4,low,med,5more,2,big,low,unacc


In [9]:
preds = []
decisions = []
for _, row in df_test.iterrows():
    # Write code to predict the class of the current set of features and append the prediction to the preds list
    # Also append the decisions made by the model to the decisions list
    
    pred, decision = clf.predict(row)
    preds.append(pred)
    decisions.append(decision)


In [10]:
# Let's check the accuracy of the model

true = df_test['Decision'].tolist()

correct = 0
for i in range(len(true)):
    if true[i] == preds[i]:
        correct += 1

accuracy = correct / len(true)

print(f'Accuracy: {accuracy}')

Accuracy: 0.9386503067484663
